In [1]:
# This notebook is dedicated to viewing outputs from APfarm evaluation, and deciding whether they match with human preferences

In [30]:
import pandas as pd
from IPython.display import clear_output
from debug_utils import load_rm
from statistics import mean

In [31]:
outdir = "../outputs/apeval/"
tmptmp = pd.read_json(outdir+"apfarmeval/da125.jsonl", lines=True, orient='records')

In [32]:
tmptmp

,instruction,input,output_1,output_2,annotator,preference,orig1,orig2,da1,da2
0,I successfully highlight the section in a web ...,,1) you must use `SendKeys.SEND_KEY` instead of...,1) It will be much easier if you use `Actions`...,chatgpt_2,1,1.286133,2.458984,1.633789,2.083984
1,"I have a json file\n\n```\n{""id"": 103, ""data"":...",,1) Try `astype` - it will do the conversion yo...,"1. To use `list()` you need an iterable, like ...",gpt4_3,2,0.039886,-0.315430,0.018478,0.971191
2,I have Qt creator 4.6.0 and Microsoft Visual S...,,1) Install Visual studio's Community Edition v...,32 bit version of `VS2017` uses `VC++ redistri...,davinci003_1,1,0.924805,1.333984,1.755859,2.099609
3,Apologies for the poorly worded question title...,,1) You should always use `__filename` and not ...,1) This:\n\n```\nmodule Foo\n def filename\n ...,chatgpt_4,2,0.080261,2.097656,0.514648,2.507812
4,Is there a word like facadal or facadeous? The...,,1. **superficial** \n\n*used to describe the...,"1. *facadal (adj) = *facadious (adj): ""like th...",gpt4_2,1,0.730469,0.618164,2.015625,1.946289
...,...,...,...,...,...,...,...,...,...,...
95,Has anyone found any documentation or research...,,1) <http://www.google.com/support/webmasters/b...,1) This page should answer your question: <htt...,gpt4_3,1,1.785156,0.653809,2.849609,1.554688
96,I could not understand the phrase *provision o...,,1. It's providing something for someone to use...,1. ***Provide*** can be used for both goods/se...,davinci003_4,1,0.474609,0.339111,1.516602,1.563477
97,When two programs are running at the same time...,,1. If the pointer was pointing somewhere outsi...,"1. If by ""print"" you mean `printf`, then yes;\...",davinci003_2,1,0.606934,0.978027,1.020508,1.309570
98,I am using Directory.GetFiles to get files fro...,,**You could use [System.IO](https://docs.micro...,\nYou could do something like this :\n\n```\nv...,gpt4_3,1,-0.108276,1.851562,0.276367,2.013672


In [5]:
tok, orm, kwargs = load_rm("../stack-llama/models/rewardsanity/", 0)
osco1s = orm(["Question: " + ex['instruction'] + "\n\nAnswer: " + ex['output_1'] for _, ex in tmp.iterrows()], **kwargs)
tmp['orig1'] = [out[0]['score'] for out in osco1s]
osco2s = orm(["Question: " + ex['instruction'] + "\n\nAnswer: " + ex['output_2'] for _, ex in tmp.iterrows()], **kwargs)
tmp['orig2'] = [out[0]['score'] for out in osco2s]
del orm
tok, darm, kwargs = load_rm("../../tfr-decoding/trlx_train/trl-stack/models/rewardda/", 0)
dsco1s = darm(["Question: " + ex['instruction'] + "\n\nAnswer: " + ex['output_1'] for _, ex in tmp.iterrows()], **kwargs)
tmp['da1'] = [out[0]['score'] for out in dsco1s]
dsco2s = darm(["Question: " + ex['instruction'] + "\n\nAnswer: " + ex['output_2'] for _, ex in tmp.iterrows()], **kwargs)
tmp['da2'] = [out[0]['score'] for out in dsco2s]
#send the new stuff back
tmp.to_json(outdir+"apfarmeval/da125.jsonl", lines=True, orient='records')

/home/prasann/miniconda3/envs/rlenv/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/prasann/miniconda3/envs/rlenv/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [33]:
# Function to filter dataframe and collect human labels
def collect_human_labels(df, substring):
    filtered_df = df[df['annotator'].str.contains(substring)]
    filtered_df = filtered_df.iloc[:25]
    print(filtered_df.preference.mean())
    human_labels = []
    print(list(filtered_df.preference))
    for idx, row in filtered_df.iterrows():
        print("_"*100)
        print(f"Question: {row['instruction']}")
        print("*"*50)
        print(f"Answer 1: {row['output_1']}")
        print("*"*50)
        print(f"Answer 2: {row['output_2']}")
        
        label = None
        #while label not in ['1', '2', '3', '4']:
        #    label = input("Which answer is better? Type '1' for Answer 1, '2' for Answer 2. 3 if equal/bad, 4 if equal/good: ").upper()
        
        human_labels.append(label)
        #clear_output(wait=True)
    
    filtered_df['human_label'] = human_labels
    
    return filtered_df

In [ ]:
humlab = collect_human_labels(tmptmp, "gpt4")

In [40]:
print(list(humlab['orig1'] - humlab['orig2']))

[0.3553161621, 0.1123046875, -0.4622192383, -0.9819335938, -0.6071777343, -0.3910217286, -0.4443359375, -0.3525390625, -0.8046875, 0.2104492187, 0.4609375, -1.9401245117, 0.18115234380000012, -1.3081054687, 0.7373046875, -0.003417968699999996, -1.4143066406, -0.0078125, 0.009765625, 0.5634765625, -0.0849609375, -1.1979980469, -0.6779785157, -0.798828125, -0.3479003906]


In [39]:
print(list(humlab['da1'] - humlab['da2']))

[-0.9527130126, 0.0693359375, -2.4506835938, -0.349609375, -0.2255859375, -0.7587890625, -0.111328125, 0.9399414062, -1.4858398437, -0.751953125, 0.64453125, -0.6557617188, 0.658203125, -1.7346191406, 0.078125, -0.5808105469, -1.88671875, -0.0166015625, -0.5732421875, -0.359375, -0.5390625, -2.1049804688, -1.4679565430000001, -1.4189453125, -0.2524414063]


In [12]:
oprefs = [1 if e.orig1>e.orig2 else 2 for _, e in humlab.iterrows()]
daprefs = [1 if e.da1>e.da2 else 2 for _, e in humlab.iterrows()]

In [13]:
oprefs

[1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2]

In [14]:
daprefs

[2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [17]:
gpts = list(humlab.preference)

In [28]:
mean(oprefs)

1.68

In [29]:
sum([gpts[i]==oprefs[i] for i in range(len(gpts))])/len(humlab)

0.6

In [ ]:
# Filter and collect labels
substring = input("Enter the substring to filter annotators: ")
new_df = collect_human_labels(df, substring)